Bioinformatics Algorithms: genome assembly

To assemble a genome from its shotgun fragments (complete collection of all possible fragments, all the same length), do the following:<br>
. turn the k-mer collection (p137_data.txt) into an adjacency list by running the code 'De Bruijn algorithm'.<br>
. use the adjacency list (p147_data.txt) to make an ordered list of the k-mers by running the code 'Euler's theorem: path'.<br>
. use the ordered list (p125_data.txt) to reconstruct the DNA string by running the code 'Un-shotgun'.

 Shotgun<br>
 Input: An integer k and a string Text.<br>
 Output: All possible k-mers that are part of Text, k(Text) (the k-mers in alphabetical order).

In [1]:
with open('p120_data.txt') as f:
    firstline = f.readline().split('\n')
    k = int(firstline[0])
    secondline = f.readline().split('\n')
    text = secondline[0]
output = []
for i in range(len(text)-k+1):
    output.append(text[i:i+k])
    output.sort()
print('\n'.join(output))

...


Un-shotgun<br>
Input: ordered k-mers (k_set) that overlap perfectly and shift one base to the right each time.<br>
Output: re-assembled shotgunned string.

In [49]:
with open('p125_data.txt') as f:
    k_set = f.read().split('\n')
f.close()
output = ''
for i in range(len(k_set)-1):
    output = output + k_set[i][0]
output = output + k_set[-1]
print(output)

CAAATGCATACGCTCATCACCCAG


Construct overlap graph<br>
Input: A collection Patterns of k-mers.<br>
Output: List (dim N nodes) of lists. The overlap graph Overlap(Patterns), in the form of an adjacency list. 

In [53]:
with open('p128_data.txt') as f:
    k_set = f.read().split('\n')
f.close()
output = []
for i in range(len(k_set)):
    sub_output = []
    compare_set = list(k_set)
    compare_set.remove(k_set[i])
    for j in range(len(compare_set)):
        if k_set[i][1:] == compare_set[j][:-1]:
            sub_output.append(compare_set[j])
    if len(sub_output) > 0:
        print(k_set[i] + ' -> ' + sub_output[0])

GCAACTGTTATTCAAATAAAACGTA -> CAACTGTTATTCAAATAAAACGTAT
AACGCCCCGGTCGCGGTGTGGCGAT -> ACGCCCCGGTCGCGGTGTGGCGATT
GGGCGTAGGAAAAAGCTTCAGTAGG -> GGCGTAGGAAAAAGCTTCAGTAGGA
TTTTTGGTTTTCATTAACACAGTAA -> TTTTGGTTTTCATTAACACAGTAAC
TTTGCAGGGTTTACACCCTCGGTAT -> TTGCAGGGTTTACACCCTCGGTATG
ATGTCTATTGAATAGCCTTCGCTCG -> TGTCTATTGAATAGCCTTCGCTCGG
ACAGTCATTCCGTGGGGGGCGTCGT -> CAGTCATTCCGTGGGGGGCGTCGTA
CGTTTGCGCGGCCAGCACGGCGAGC -> GTTTGCGCGGCCAGCACGGCGAGCA
TCTATCACGTCTAAGATAGCTTGGT -> CTATCACGTCTAAGATAGCTTGGTG
TCGCCCTGTTGCTCTGGGATTGGTA -> CGCCCTGTTGCTCTGGGATTGGTAC
ATCAAGAATTGCTCGCTTCGACGCT -> TCAAGAATTGCTCGCTTCGACGCTG
TCCTGAGCCGCATCACGGTTAATCA -> CCTGAGCCGCATCACGGTTAATCAG
TTTACCATACAGCATTACTCGGCCG -> TTACCATACAGCATTACTCGGCCGA
ACTCTCTAGCATTTACCAGAATTAC -> CTCTCTAGCATTTACCAGAATTACA
TCCAAATTTGCATTTGGTACGAGGC -> CCAAATTTGCATTTGGTACGAGGCC
CCTTCGCTCGGATACTACTGAGTAC -> CTTCGCTCGGATACTACTGAGTACT
AGATGTCATCAATTAAGCGGGCAGA -> GATGTCATCAATTAAGCGGGCAGAC
GATAACGGGTTTACCCGCAGTATCG -> ATAACGGGTTTACCCGCAGTATCGC
AAAGAAGATT

TTGTATACCACTTTATCTAGAACTT -> TGTATACCACTTTATCTAGAACTTC
ATCATCGTACCTAGTCGTACCTAGC -> TCATCGTACCTAGTCGTACCTAGCT
TTTCCGTGGAAATACCCGCAGAGGT -> TTCCGTGGAAATACCCGCAGAGGTA
CCCTATCGTTGGATGCAACGCAGGA -> CCTATCGTTGGATGCAACGCAGGAT
CATCAATTAAGCGGGCAGACGCAGG -> ATCAATTAAGCGGGCAGACGCAGGG
ACAAACTAAACAGTTACATTATGAG -> CAAACTAAACAGTTACATTATGAGG
ACGCGGACTACCATTCATTAGTGCG -> CGCGGACTACCATTCATTAGTGCGG
TTGAGCTGGACCAGACAGTTCGGCA -> TGAGCTGGACCAGACAGTTCGGCAT
ACCTGACCGCTAGGTACTGGCACCG -> CCTGACCGCTAGGTACTGGCACCGA
GAATAGGTGGACAAACTAAACAGTT -> AATAGGTGGACAAACTAAACAGTTA
CGCCCCGGTCGCGGTGTGGCGATTG -> GCCCCGGTCGCGGTGTGGCGATTGT
ACTGTTCGCTCTCAGCATTATGGTT -> CTGTTCGCTCTCAGCATTATGGTTT
AACGGAGGCTTTACATGATTCGTGA -> ACGGAGGCTTTACATGATTCGTGAT
TGCGTGCCCCCACATCACAGGACCT -> GCGTGCCCCCACATCACAGGACCTA
GAGCTTCAATCCTCGAGAAGGGACG -> AGCTTCAATCCTCGAGAAGGGACGC
CCAGTCTGCTCTTGGGAGGTGTTCA -> CAGTCTGCTCTTGGGAGGTGTTCAT
GTATTTCAGTGCGAACAGTATCCAC -> TATTTCAGTGCGAACAGTATCCACT
GCACGCTGGCCCCCGCAGGCGCTAA -> CACGCTGGCCCCCGCAGGCGCTAAT
CAAGCCGCAA

GTCCTATGAGCTAAAACCAAATCCT -> TCCTATGAGCTAAAACCAAATCCTT
TCGCCTCTCTAGATTCAAGGTATAC -> CGCCTCTCTAGATTCAAGGTATACG
GGGACGCCTGCGAGGACGCCGTGCT -> GGACGCCTGCGAGGACGCCGTGCTC
CGACGCTGTGAAAGTAGAATCCGGG -> GACGCTGTGAAAGTAGAATCCGGGC
TGAACAGTAAAACTGTTGATCACTA -> GAACAGTAAAACTGTTGATCACTAC
CATGCTCGGAAGAGCAAATATCATG -> ATGCTCGGAAGAGCAAATATCATGT
CCTGGGATCATCGTACCTAGTCGTA -> CTGGGATCATCGTACCTAGTCGTAC
CAGGTCTCGCGACTACGAGTACACC -> AGGTCTCGCGACTACGAGTACACCG
TCTTTACTAGAAGGTAGAAGCTGAG -> CTTTACTAGAAGGTAGAAGCTGAGA
GCGATTGGAGGGTAGAGTCTACTTG -> CGATTGGAGGGTAGAGTCTACTTGG
TATCAATATAGTCGCGCAGACGGAA -> ATCAATATAGTCGCGCAGACGGAAC
GTATCGAGCAATAGTGAGCTGATCA -> TATCGAGCAATAGTGAGCTGATCAG
TTGGGAGCACCGCCTTGACGTCTTT -> TGGGAGCACCGCCTTGACGTCTTTC
CACCCTGCGACACAGACGTGTGCAC -> ACCCTGCGACACAGACGTGTGCACG
GAGCATGCTCGGAAGAGCAAATATC -> AGCATGCTCGGAAGAGCAAATATCA
AACACGACAAGCACAGTTCGCCTCG -> ACACGACAAGCACAGTTCGCCTCGC
CCTGTCAATGTTATTAGGAATCTCA -> CTGTCAATGTTATTAGGAATCTCAT
GGCTTATAGCGTGCCTCGCTGGCCT -> GCTTATAGCGTGCCTCGCTGGCCTG
GTAAAACTGT

TCTCTAGATTCAAGGTATACGTCCG -> CTCTAGATTCAAGGTATACGTCCGT
CGTATTTCAGTGCGAACAGTATCCA -> GTATTTCAGTGCGAACAGTATCCAC
TTTCTGACTAGTTACAGAGCCTGCG -> TTCTGACTAGTTACAGAGCCTGCGA
ACGGAAAGCAGAATGGTGTTCACTA -> CGGAAAGCAGAATGGTGTTCACTAG
CGTCCCGGTAATGCTCCTCGGATAG -> GTCCCGGTAATGCTCCTCGGATAGC
TTTGTGCCCGGACTGTAAGAAGCGT -> TTGTGCCCGGACTGTAAGAAGCGTT
TGAGATGCGTGCCCCCACATCACAG -> GAGATGCGTGCCCCCACATCACAGG
GTTTACACCCTCGGTATGGTGCTCG -> TTTACACCCTCGGTATGGTGCTCGT
TAAACAGTTACATTATGAGGCCAAC -> AAACAGTTACATTATGAGGCCAACA
ACTAGAAGGTAGAAGCTGAGATACA -> CTAGAAGGTAGAAGCTGAGATACAT
TTCCGACTATGACTCTCTTCTGATC -> TCCGACTATGACTCTCTTCTGATCC
TTAGGGAACAAATAAGCGACATTGA -> TAGGGAACAAATAAGCGACATTGAG
TCAGGAATGGTCGGCACGCTACGCC -> CAGGAATGGTCGGCACGCTACGCCT
TCACAGGACCTATGATCGCCCAGCG -> CACAGGACCTATGATCGCCCAGCGT
GTGGTGATCTCCTTACTATATGGAG -> TGGTGATCTCCTTACTATATGGAGG
CCGCGCCTGCAACAGGTCTCGCGAC -> CGCGCCTGCAACAGGTCTCGCGACT
CAATTCCTGGCCGCTTGCGTTGCGC -> AATTCCTGGCCGCTTGCGTTGCGCC
GGTCAGGTCTACCTTTATTGGCTGA -> GTCAGGTCTACCTTTATTGGCTGAT
TTCCTGGTAT

ACCTTTACCATACAGCATTACTCGG -> CCTTTACCATACAGCATTACTCGGC
GCTGGCCTGGGACGCCTGCGAGGAC -> CTGGCCTGGGACGCCTGCGAGGACG
CTGTCATTTATCCGTTCCTGTCAAT -> TGTCATTTATCCGTTCCTGTCAATG
TAACATAACCATGGTTTTTCTGTGT -> AACATAACCATGGTTTTTCTGTGTA
GCGCTCGTACTGTTGGGAGCACCGC -> CGCTCGTACTGTTGGGAGCACCGCC
CTCGGAAGAGCAAATATCATGTCTA -> TCGGAAGAGCAAATATCATGTCTAC
TCCTAGGGGACGCTATACCTTTACC -> CCTAGGGGACGCTATACCTTTACCA
CGAAATAATTTCCGACTAAAGTAAC -> GAAATAATTTCCGACTAAAGTAACG
ACCTAGTCGTACCTAGCTATCAAGT -> CCTAGTCGTACCTAGCTATCAAGTT
TTCAAATAAAACGTATTGGCCTCTA -> TCAAATAAAACGTATTGGCCTCTAC
CGTTTGCCGATTCACAAGCGTTTAG -> GTTTGCCGATTCACAAGCGTTTAGA
GGGTATACCAGAGAGGAGTCCGAAT -> GGTATACCAGAGAGGAGTCCGAATC
TCGCGACGACAGACCGTAACGGTTG -> CGCGACGACAGACCGTAACGGTTGG
AATTGCTCGCTTCGACGCTGTGAAA -> ATTGCTCGCTTCGACGCTGTGAAAG
CATTCATTAGTGCGGGTCAGGTCTA -> ATTCATTAGTGCGGGTCAGGTCTAC
CAATCTATGAGAAAATCAGCCTTCG -> AATCTATGAGAAAATCAGCCTTCGC
ACCCCGCAAAAAACAAAAGAAGATT -> CCCCGCAAAAAACAAAAGAAGATTA
GCGTGCCTCGCTGGCCTGGGACGCC -> CGTGCCTCGCTGGCCTGGGACGCCT
TCGCCATCGG

...

TAACATCAGCGAGGTGCCTTGAGAT -> AACATCAGCGAGGTGCCTTGAGATG
AATGCTCCTCGGATAGCTTCCTGGG -> ATGCTCCTCGGATAGCTTCCTGGGA
AACGAGCACGAATAAGACCTGACCG -> ACGAGCACGAATAAGACCTGACCGC
GCTTAAAAGCATCCGAGATCGTGCC -> CTTAAAAGCATCCGAGATCGTGCCC
ACGACAAGCACAGTTCGCCTCGCAA -> CGACAAGCACAGTTCGCCTCGCAAT
TTCTTGGATACGTCGGACACAGCAA -> TCTTGGATACGTCGGACACAGCAAA
GATCTCCTTACTATATGGAGGCCAA -> ATCTCCTTACTATATGGAGGCCAAA
CCACTTAGACATATACGCCCCAGTC -> CACTTAGACATATACGCCCCAGTCC
GGTCGGGCAGCATTTGCTCTCTTCT -> GTCGGGCAGCATTTGCTCTCTTCTT
TCGGCGATTGGAGGGTAGAGTCTAC -> CGGCGATTGGAGGGTAGAGTCTACT
CCGGCGTGGTGATCTCCTTACTATA -> CGGCGTGGTGATCTCCTTACTATAT
AGCACGGCGAGCACAGTGATAACAA -> GCACGGCGAGCACAGTGATAACAAA
GCTCTCTTCTTAGTCAACCTTGCAT -> CTCTCTTCTTAGTCAACCTTGCATC
GTGACAACTGCCTTCCAGACAGAGG -> TGACAACTGCCTTCCAGACAGAGGG
GACACATTCATAAGAGAGGCAGTAT -> ACACATTCATAAGAGAGGCAGTATC
GTCTCAACCCGCTGATACTACGCGG -> TCTCAACCCGCTGATACTACGCGGA
CTCGAGAAGGGACGCACGCTGGCCC -> TCGAGAAGGGACGCACGCTGGCCCC
GCAGCATTTGCTCTCTTCTTAGTCA -> CAGCATTTGCTCTCTTCTTAGTCAA
GACTTTACGC

CATATGTCTATTGAATAGCCTTCGC -> ATATGTCTATTGAATAGCCTTCGCT
AGATACCAACGCTGTTTTGACCGAT -> GATACCAACGCTGTTTTGACCGATC
AGCACCTCTCTGCTTTGTCTATGTT -> GCACCTCTCTGCTTTGTCTATGTTA
AGAAATAACCCCGCAAAAAACAAAA -> GAAATAACCCCGCAAAAAACAAAAG
TCGCCTCGCAATTCCAGTCTGCTCT -> CGCCTCGCAATTCCAGTCTGCTCTT
TGATCCAATCTATGAGAAAATCAGC -> GATCCAATCTATGAGAAAATCAGCC
GCAAAAATTGCGTACCCGTCAATTC -> CAAAAATTGCGTACCCGTCAATTCC
TCGTGATAGGCGGACCTCAGGTGGC -> CGTGATAGGCGGACCTCAGGTGGCA
ACAGGACCTATGATCGCCCAGCGTC -> CAGGACCTATGATCGCCCAGCGTCC
ATGATCCCAGCTGATCTATGCTGAC -> TGATCCCAGCTGATCTATGCTGACC
GGCTAAGTAGACGTGCATCTTGACA -> GCTAAGTAGACGTGCATCTTGACAA
AATCTTGGACATAAGTTATGAATGA -> ATCTTGGACATAAGTTATGAATGAT
ACTCTCTTCTGATCCAATCTATGAG -> CTCTCTTCTGATCCAATCTATGAGA
CTATCACGTCTAAGATAGCTTGGTG -> TATCACGTCTAAGATAGCTTGGTGG
AGTCAACCTTGCATCAGACAGACTG -> GTCAACCTTGCATCAGACAGACTGT
CTAAGTAGACGTGCATCTTGACAAC -> TAAGTAGACGTGCATCTTGACAACG
ACGACATCGAGACTGTAAAATACCA -> CGACATCGAGACTGTAAAATACCAA
CACAGCAAAGATGGGAACGGTAACA -> ACAGCAAAGATGGGAACGGTAACAG
CGCTAATTAG

TAACAGTTCCTGGCCTAACCACGTC -> AACAGTTCCTGGCCTAACCACGTCC
GCCCTAACGTATTTCGGTGGCCTCG -> CCCTAACGTATTTCGGTGGCCTCGT
CGCGGCTAAGTAGACGTGCATCTTG -> GCGGCTAAGTAGACGTGCATCTTGA
AGCGCTCGTACTGTTGGGAGCACCG -> GCGCTCGTACTGTTGGGAGCACCGC
TGCACCCTCGCCATCGGGACCGTTG -> GCACCCTCGCCATCGGGACCGTTGA
GAGGCCAACAGGCTCCTGCCATATA -> AGGCCAACAGGCTCCTGCCATATAT
CATTATGCTTCCGACTATGACTCTC -> ATTATGCTTCCGACTATGACTCTCT
GTTATGAATGATACTAGCGTTGTCT -> TTATGAATGATACTAGCGTTGTCTG
GGCGTCCTCAATAACTTACTTACAC -> GCGTCCTCAATAACTTACTTACACA
GGAGTCCGAATCGCCTCTCTAGATT -> GAGTCCGAATCGCCTCTCTAGATTC
TACAGGCAGCAACAGGTGCCATAAC -> ACAGGCAGCAACAGGTGCCATAACC
AGAAGCGTTCTCCATTACTCTCTAG -> GAAGCGTTCTCCATTACTCTCTAGC
CTGTTACATTCGCTGAGTAACAGTA -> TGTTACATTCGCTGAGTAACAGTAC
ACGCTGAAGAAATAACCCCGCAAAA -> CGCTGAAGAAATAACCCCGCAAAAA
ATTGCTCGCTTCGACGCTGTGAAAG -> TTGCTCGCTTCGACGCTGTGAAAGT
ATCTTGTACGATTGTATGTATCCAA -> TCTTGTACGATTGTATGTATCCAAG
GGCGGGTATACCAGAGAGGAGTCCG -> GCGGGTATACCAGAGAGGAGTCCGA
TCCTCGGATAGCTTCCTGGGATCAT -> CCTCGGATAGCTTCCTGGGATCATC
GGGGATCGCT

ACTAGTTACAGAGCCTGCGATCAGT -> CTAGTTACAGAGCCTGCGATCAGTG
TGCTCGTCTGTCCGCAATGTCTTTT -> GCTCGTCTGTCCGCAATGTCTTTTT
TTTCGTACGCAGAGTACTGCAGCTG -> TTCGTACGCAGAGTACTGCAGCTGG
AGACGCAGGGTATGTAACTGGGGGT -> GACGCAGGGTATGTAACTGGGGGTT
GGGGCCCTCCAAGCCGCAACCAGCC -> GGGCCCTCCAAGCCGCAACCAGCCT
TGTGCTACCGTTCGCGACGACAGAC -> GTGCTACCGTTCGCGACGACAGACC
TACCAACGCTGTTTTGACCGATCTT -> ACCAACGCTGTTTTGACCGATCTTG
AGAAAAACGTATTTCAGTGCGAACA -> GAAAAACGTATTTCAGTGCGAACAG
GTGTGGGACACGGCAGCATGTTTGA -> TGTGGGACACGGCAGCATGTTTGAT
CCTAGTTAAGATGCGACGTACCAAG -> CTAGTTAAGATGCGACGTACCAAGG
GCTTAAAAAACACGACAAGCACAGT -> CTTAAAAAACACGACAAGCACAGTT
GGAACCATAGAAGTAGTTCTGTCAT -> GAACCATAGAAGTAGTTCTGTCATT
CGTTGGATGCAACGCAGGATTCCTG -> GTTGGATGCAACGCAGGATTCCTGG
ATACGCCCCAGTCCCACCATATTAA -> TACGCCCCAGTCCCACCATATTAAA
AGCGCCCGGTGCGGTCAGCGGGGAC -> GCGCCCGGTGCGGTCAGCGGGGACG
CTTCTTAGTCAACCTTGCATCAGAC -> TTCTTAGTCAACCTTGCATCAGACA
CAGACTAAACATTTATTTATACCAG -> AGACTAAACATTTATTTATACCAGG
AACTAGCACCTCTCTGCTTTGTCTA -> ACTAGCACCTCTCTGCTTTGTCTAT
TGGTCGTAGC

TCATCGTATAGTCCGAAGGAAATAG -> CATCGTATAGTCCGAAGGAAATAGA
TGCGAACAGTATCCACTCCCAGACA -> GCGAACAGTATCCACTCCCAGACAC
AGAACTTCCTCCTATACAGAGACTA -> GAACTTCCTCCTATACAGAGACTAC
GCAGCATGTTTGATTAGAGGCGAAC -> CAGCATGTTTGATTAGAGGCGAACT
AACATCAGCGAGGTGCCTTGAGATG -> ACATCAGCGAGGTGCCTTGAGATGC
TACCGTGAGCCGGTTTGCATGCCGC -> ACCGTGAGCCGGTTTGCATGCCGCC
GGCAGTATCCGGCGACAATGTGATG -> GCAGTATCCGGCGACAATGTGATGT
ACCGATGACGCGACCGACCTCACAG -> CCGATGACGCGACCGACCTCACAGT
ATCGTTGAGCTGGACCAGACAGTTC -> TCGTTGAGCTGGACCAGACAGTTCG
CCGCTGATACTACGCGGACTACCAT -> CGCTGATACTACGCGGACTACCATT
TTTGCTCTCTTCTTAGTCAACCTTG -> TTGCTCTCTTCTTAGTCAACCTTGC
ACCCTAGAACTGTTGTCTGTCTACA -> CCCTAGAACTGTTGTCTGTCTACAG
AACCGTATTTTTCGAAAGGAAATAT -> ACCGTATTTTTCGAAAGGAAATATT
CCGCAGGCGCTAATTAGTACCACAC -> CGCAGGCGCTAATTAGTACCACACG
CGCTCCCAACGGAAAGCAGAATGGT -> GCTCCCAACGGAAAGCAGAATGGTG
ATTTCCGACTAAAGTAACGTAAGAG -> TTTCCGACTAAAGTAACGTAAGAGC
TTAGTCGTTGATAACGGGTTTACCC -> TAGTCGTTGATAACGGGTTTACCCG
TTGTCCCCTGAGCTCGCCGTAAACA -> TGTCCCCTGAGCTCGCCGTAAACAC
TTTTAGTCGT

De Bruijn graph<br>
Input: An integer k and a string Text.<br>
Output: DeBruijnk(Text), in the form of an adjacency list (dictionary).

In [92]:
import collections
with open('p132_data.txt') as f:
    k = int(f.readline())
    text = f.readline().strip('\n')
f.close()
nodes = []
bruijn = {}
for i in range(len(text)-k+2):
    node = text[i:i+k-1]
    nodes.append(node)
for j in range(len(nodes)-1):
    if nodes[j] in bruijn:
        bruijn[nodes[j]].append(nodes[j+1])
    else:
        bruijn[nodes[j]] = [nodes[j+1]]
o_bruijn = collections.OrderedDict(sorted(bruijn.items()))
for key, value in o_bruijn.items():
    print(key + ' -> ' + ','.join(value))

AA -> AT
AT -> TG,TG,TG
CA -> AT
CC -> CA
GA -> AT
GC -> CC
GG -> GG,GA
GT -> TT
TA -> AA
TG -> GG,GC,GT


De Bruijn algorithm<br>
Input: A collection of k-mers Patterns.<br>
Output: The adjacency list of the de Bruijn graph DeBruijn(Patterns).

In [47]:
import collections
with open('p137_data.txt') as f:
    patterns = f.read().split('\n')
f.close()
nodes = []
bruijn = {}
k=len(patterns[0])
for j in range(len(patterns)):
    node = patterns[j][:k-1]
    if node in bruijn:
        bruijn[node].append(patterns[j][-(k-1):])
    else:
        bruijn[node] = [patterns[j][-(k-1):]]
o_bruijn = collections.OrderedDict(sorted(bruijn.items()))
for key, value in o_bruijn.items():
    value.sort()
    print(key + ' -> ' + ','.join(value))

AAA -> AAT
AAT -> ATG
ACC -> CCC
ACG -> CGC
ATA -> TAC
ATC -> TCA
ATG -> TGC
CAA -> AAA
CAC -> ACC
CAT -> ATA,ATC
CCA -> CAG
CCC -> CCA
CGC -> GCT
CTC -> TCA
GCA -> CAT
GCT -> CTC
TAC -> ACG
TCA -> CAC,CAT
TGC -> GCA


Euler's theorem: cycle<br>
Input: The adjacency list of an Eulerian directed graph.<br>
Output: An Eulerian cycle in this graph (6->8->7->9->6->5->4->2->1->0->3->2->6).

In [12]:
import re
bruijn = {}
with open('p146_data.txt') as f:
    for line in f:
        numbers = re.findall('([0-9]+)',line)
        key = numbers[0]
        values = numbers[1:]
        for i in range(len(values)):
            v = values[i]
            values[i] = v
        bruijn[key] = values
f.close()
bruijn_leftover = bruijn
cycle = []
start = list(bruijn.keys())[0]
cycle.append(start)
cycle_next_step = bruijn[start][0]
current_position = start

#find the first cycle
while cycle_next_step != start:
    cycle.append(cycle_next_step)
    current_position = cycle_next_step
    cycle_next_step = bruijn[current_position][0]
    bruijn_leftover[cycle[-2]].remove(current_position)
bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
unexplored_edges = []
for key in bruijn_leftover:
    unexplored_edges.extend(bruijn_leftover[key])

#find the other cycles
while len(unexplored_edges) > 1:
    #find an unexplored edge in cycle
    for edge in cycle:
        if len(bruijn_leftover[edge]) > 0:
            new_start = edge
    #re-order cycle to start at new_start
    cycle = cycle[cycle.index(new_start):] + cycle[:cycle.index(new_start)]
    #unexplored_edges.remove(new_start)
    current_position = new_start
    unexplored_edges.remove(new_start)
    cycle.append(current_position)
    cycle_next_step = bruijn_leftover[current_position][0]
    while cycle_next_step != new_start:
        cycle.append(cycle_next_step)
        unexplored_edges.remove(cycle_next_step)
        current_position = cycle_next_step
        cycle_next_step = bruijn_leftover[current_position][0]
        bruijn_leftover[cycle[-2]].remove(current_position)
    bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
cycle.append(cycle_next_step)
#print(bruijn_leftover)
output = []
for j in range(len(cycle)):
    output.append(str(cycle[j]))
#print('->'.join(output))
print('\n'.join(output))

11111110
11111100
11111000
11110000
11100000
11000000
10000000
00000000
00000000
00000001
00000010
00000100
00001000
00010000
00100000
01000000
10000000
00000001
00000011
00000110
00001100
00011000
00110000
01100000
11000000
10000001
00000010
00000101
00001010
00010100
00101000
01010000
10100000
01000000
10000001
00000011
00000111
00001110
00011100
00111000
01110000
11100000
11000001
10000010
00000100
00001001
00010010
00100100
01001000
10010000
00100000
01000001
10000010
00000101
00001011
00010110
00101100
01011000
10110000
01100000
11000001
10000011
00000110
00001101
00011010
00110100
01101000
11010000
10100000
01000001
10000011
00000111
00001111
00011110
00111100
01111000
11110000
11100001
11000010
10000100
00001000
00010001
00100010
01000100
10001000
00010000
00100001
01000010
10000100
00001001
00010011
00100110
01001100
10011000
00110000
01100001
11000010
10000101
00001010
00010101
00101010
01010100
10101000
01010000
10100001
01000010
10000101
00001011
00010111
00101110
01011100
1

Euler's theorem: path<br>
Input: The adjacency list of a directed graph that has an Eulerian path.<br>
Output: An Eulerian path in this graph  6->7->8->9->6->3->0->2->1->3->4.<br>

In [48]:
import re
bruijn = {}
with open('p147_data.txt') as f:
    for line in f:
        numbers = re.findall('([0-9A-Z]+)',line)
        key = numbers[0]
        values = numbers[1:]
        for i in range(len(values)):
            v = values[i]
            values[i] = v
        bruijn[key] = values
f.close()
cycle = []

#find the in- and outnodes; they are unbalanced, i.e., the number of indegrees != number of outdegrees
balance = {}
for key in bruijn:
    if key in balance:
        balance[key] = balance[key] + len(bruijn[key])
    else:
        balance[key] = len(bruijn[key])
    for j in range(len(bruijn[key])):
        if bruijn[key][j] in balance:
            balance[bruijn[key][j]] = balance[bruijn[key][j]] - 1
        else:
            balance[bruijn[key][j]] = -1
for key in balance:
    if balance[key] == 1: beginning = key
    if balance[key] == -1: end = key

#connect the unbalanced nodes
if end in bruijn:
    bruijn[end].append(beginning)
else:
    bruijn[end] = [beginning]
    
#find the first cycle
bruijn_leftover = bruijn
cycle = []
start = beginning
cycle.append(start)
cycle_next_step = bruijn[start][0]
current_position = start
while cycle_next_step != start:
    cycle.append(cycle_next_step)
    current_position = cycle_next_step
    cycle_next_step = bruijn[current_position][0]
    bruijn_leftover[cycle[-2]].remove(current_position)
bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
unexplored_edges = []
for key in bruijn_leftover:
    unexplored_edges.extend(bruijn_leftover[key])
    
#find the other cycles
while len(unexplored_edges) > 1:
    #find an unexplored edge in cycle
    for edge in cycle:
        if len(bruijn_leftover[edge]) > 0:
            new_start = edge
    #re-order cycle to start at new_start
    cycle = cycle[cycle.index(new_start):] + cycle[:cycle.index(new_start)]
    #unexplored_edges.remove(new_start)
    current_position = new_start
    unexplored_edges.remove(new_start)
    cycle.append(current_position)
    cycle_next_step = bruijn_leftover[current_position][0]
    while cycle_next_step != new_start:
        cycle.append(cycle_next_step)
        unexplored_edges.remove(cycle_next_step)
        current_position = cycle_next_step
        cycle_next_step = bruijn_leftover[current_position][0]
        bruijn_leftover[cycle[-2]].remove(current_position)
    bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
cycle.append(cycle_next_step)

#re-order cycle by cutting after end
if cycle[-1] != end:
    for c in range(len(cycle)-1):
        if cycle[c] == end and cycle[c+1] == beginning: cutting_point = c+1
    cycle = cycle[cutting_point:-1] + cycle[:cutting_point]

#print(bruijn_leftover)
output = []
for j in range(len(cycle)):
    output.append(str(cycle[j]))
print('\n'.join(output))

CAA
AAA
AAT
ATG
TGC
GCA
CAT
ATA
TAC
ACG
CGC
GCT
CTC
TCA
CAT
ATC
TCA
CAC
ACC
CCC
CCA
CAG


k-Universal binary circular string<br>
Input: An integer k.<br>
Output: A k-universal binary circular string.

In [10]:
#produce all possible k-mers and proceed as described at the top
import itertools
k = 9
collection = list(itertools.product('01', repeat=k))
for i in range(len(collection)):
    print(''.join(collection[i]))

000000000
000000001
000000010
000000011
000000100
000000101
000000110
000000111
000001000
000001001
000001010
000001011
000001100
000001101
000001110
000001111
000010000
000010001
000010010
000010011
000010100
000010101
000010110
000010111
000011000
000011001
000011010
000011011
000011100
000011101
000011110
000011111
000100000
000100001
000100010
000100011
000100100
000100101
000100110
000100111
000101000
000101001
000101010
000101011
000101100
000101101
000101110
000101111
000110000
000110001
000110010
000110011
000110100
000110101
000110110
000110111
000111000
000111001
000111010
000111011
000111100
000111101
000111110
000111111
001000000
001000001
001000010
001000011
001000100
001000101
001000110
001000111
001001000
001001001
001001010
001001011
001001100
001001101
001001110
001001111
001010000
001010001
001010010
001010011
001010100
001010101
001010110
001010111
001011000
001011001
001011010
001011011
001011100
001011101
001011110
001011111
001100000
001100001
001100010
001100011


Solve the String Reconstruction from Read-Pairs Problem.<br>
Input: Integers k and d followed by a collection of paired k-mers PairedReads.<br>
Output: A string Text with (k, d)-mer composition equal to PairedReads.

In [52]:
import re
first_patterns = []
second_patterns = []
with open('p149_data.txt') as f:
    parameters = f.readline().strip()
    k = int(re.findall('([0-9]+)',parameters)[0])
    d = int(re.findall('([0-9]+)',parameters)[1])
    for line in f:
        line.strip()
        numbers = re.findall('([0-9A-Z]+)',line)
        values = numbers
        for i in range(len(values)):
            v = values[i]
            values[i] = v
        first_patterns.append(values[0])
        second_patterns.append(values[1])
f.close()

graph = de_bruijn_algorithm_for_pairs(first_patterns,second_patterns)
ordered_kmers = eulers_theorem_path(graph)
genome = unshotgun_paired(ordered_kmers)
print(genome)

#CACCGATACTGATTCTGAAGCTT

HELLO CAGA AATT
OUTPUT ['CAGA', 'ACAT', 'CCTA', 'CGAC', 'GACT', 'ATTG', 'TTGA', 'TCAA', 'CTAG', 'TGGC', 'GACT', 'ATTG', 'TAGA', 'ACAT', 'CTTT', 'TGTC', 'GACT', 'AATT']
['CAGA', 'ACAT', 'CCTA', 'CGAC', 'GACT', 'ATTG', 'TTGA', 'TCAA', 'CTAG', 'TGGC', 'GACT', 'ATTG', 'TAGA', 'ACAT', 'CTTT', 'TGTC', 'GACT', 'AATT']
CACCGATTCTGATACTGAA GATACTGAAGCTGATTCTT
A T
A A
G G
T T
C C
A G
T A
A A
G G
T T
C C
T A
T T
A A
G G
CACCGATTCTGATACTGAATCTT


In [2]:
def de_bruijn_algorithm_for_pairs(kmers1,kmers2):
    import collections
    bruijn = {}
    k=len(kmers1[0])
    for j in range(len(kmers1)):
        key = kmers1[j][:-1] + kmers2[j][:-1]
        value = [kmers1[j][1:] + kmers2[j][1:]]
        if key in bruijn:
            bruijn[key].extend(value)
        else:
            bruijn[key] = value
    o_bruijn = collections.OrderedDict(sorted(bruijn.items()))
    return o_bruijn

In [2]:
#code p137 as a function
def de_bruijn_algorithm(kmers):
    import collections
    bruijn = {}
    k=len(kmers[0])
    for j in range(len(kmers)):
        node = kmers[j][:k-1]
        if node in bruijn:
            bruijn[node].append(kmers[j][-(k-1):])
        else:
            bruijn[node] = [kmers[j][-(k-1):]]
    o_bruijn = collections.OrderedDict(sorted(bruijn.items()))
    return o_bruijn

In [24]:
#code p147 as a function
def eulers_theorem_path(ordered_dict):
    import re
    import collections
    bruijn = dict()
    for key, value in ordered_dict.items():
        value.sort()
        bruijn[key] = value
    cycle = []

    #find the in- and outnodes; they are unbalanced, i.e., the number of indegrees != number of outdegrees
    balance = {}
    for key in bruijn:
        if key in balance:
            balance[key] = balance[key] + len(bruijn[key])
        else:
            balance[key] = len(bruijn[key])
        for j in range(len(bruijn[key])):
            if bruijn[key][j] in balance:
                balance[bruijn[key][j]] = balance[bruijn[key][j]] - 1
            else:
                balance[bruijn[key][j]] = -1
    for key in balance:
        if balance[key] == 1: beginning = key
        if balance[key] == -1: end = key
    
    #connect the unbalanced nodes
    if end in bruijn:
        bruijn[end].append(beginning)
    else:
        bruijn[end] = [beginning]
    print('HELLO',beginning,end)
    
    #find the first cycle
    bruijn_leftover = bruijn
    cycle = []
    start = beginning
    cycle.append(start)
    cycle_next_step = bruijn[start][0]
    current_position = start
    while cycle_next_step != start:
        cycle.append(cycle_next_step)
        current_position = cycle_next_step
        cycle_next_step = bruijn[current_position][0]
        bruijn_leftover[cycle[-2]].remove(current_position)
    bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
    unexplored_edges = []
    for key in bruijn_leftover:
        unexplored_edges.extend(bruijn_leftover[key])

    #find the other cycles
    while len(unexplored_edges) > 1:
        #find an unexplored edge in cycle
        for edge in cycle:
            if len(bruijn_leftover[edge]) > 0:
                new_start = edge
        #re-order cycle to start at new_start
        cycle = cycle[cycle.index(new_start):] + cycle[:cycle.index(new_start)]
        #unexplored_edges.remove(new_start)
        current_position = new_start
        unexplored_edges.remove(new_start)
        cycle.append(current_position)
        cycle_next_step = bruijn_leftover[current_position][0]
        while cycle_next_step != new_start:
            cycle.append(cycle_next_step)
            unexplored_edges.remove(cycle_next_step)
            current_position = cycle_next_step
            cycle_next_step = bruijn_leftover[current_position][0]
            bruijn_leftover[cycle[-2]].remove(current_position)
        bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
    cycle.append(cycle_next_step)

    #re-order cycle by cutting after end
    if cycle[-1] != end:
        for c in range(len(cycle)-1):
            if cycle[c] == end and cycle[c+1] == beginning: cutting_point = c+1
        cycle = cycle[cutting_point:-1] + cycle[:cutting_point]

    #print(bruijn_leftover)
    output = []
    for j in range(len(cycle)):
        output.append(str(cycle[j]))
    print('OUTPUT',output)
    return output

In [4]:
#code p125 as a function
def unshotgun(ordered_kmers):
    output = ''
    for i in range(len(ordered_kmers)-1):
        output = output + ordered_kmers[i][0]
    output = output + ordered_kmers[-1]
    return output

All Eulerian cycles in a graph<br>
Input: a non-simple graph (as an OrderedDict)<br>
Output: a collection of Eulerian cycles

In [51]:
def unshotgun_paired(ordered_pairs):
    print(ordered_pairs)
    prefix = ''
    suffix = ''
    #convert back to paired data
    paired_data = []
    prefix = ordered_pairs[0][:k-1]
    suffix = ordered_pairs[0][-k+1:]
    for i in range(len(ordered_pairs)-1):
        prefix = prefix + ordered_pairs[i+1][k-2]
        suffix = suffix + ordered_pairs[i+1][-1]
    print(prefix,suffix)
    for j in range(len(suffix)-(k+d)):
        print(prefix[-(j+1)] , suffix[-(k+d+j+1)])
    return prefix + suffix[-(k+d):]

MaximalNonBranchingPaths.<br>
Input: The adjacency list of a graph whose nodes are integers.<br>
Output: The collection of all maximal nonbranching paths in this graph.

In [37]:
adjacency_dict = {1:[2],2:[3],3:[4,5],6:[7],7:[6]}
#def max_non_branching_paths(adj_list):
adjacency_dict_remaining = adjacency_dict
max_non_branching_paths = []
active_key = list(adjacency_dict_remaining.keys())[0]
while len(adjacency_dict_remaining) > 0:
    b_path = []
    if len(adjacency_dict_remaining[active_key]) == 1:
        b_path.append(active_key)
        b_path.append(adjacency_dict_remaining[active_key][0])
        print(b_path)
        if adjacency_dict_remaining[active_key][0] in adjacency_dict_remaining:
            active_key = adjacency_dict_remaining[active_key]
            print('ONE')
        else:
            del adjacency_dict_remaining[active_key]
            active_key = list(adjacency_dict_remaining.keys())[0]
            print('TWO')
    else:
        active_key = list(adjacency_dict_remaining.keys())[0]
    print(b_path)
    max_non_branching_paths.append(b_path)
print(max_non_branching_paths)

[1, 2]
ONE
[1, 2]


TypeError: unhashable type: 'list'

In [3]:
kmers = []
with open('p150_data.txt') as f:
    for line in f:
        kmers.append(line.strip())
print(kmers)
o_bruijn=(de_bruijn_algorithm(kmers))
print(o_bruijn)

['ATG', 'ATG', 'TGT', 'TGG', 'CAT', 'GGA', 'GAT', 'AGA']
OrderedDict([('AG', ['GA']), ('AT', ['TG', 'TG']), ('CA', ['AT']), ('GA', ['AT']), ('GG', ['GA']), ('TG', ['GT', 'GG'])])
